# Check for new symbols or if changed to inactive
This notebook will check for new or inactive symbols

In [2]:
# Import libraries
import binance
import pandas as pd
import ccxt
import logging
import datetime
import tqdm
import time
import os
import traceback
import sqlite3
import pdcast as pdc
from dotenv import load_dotenv


In [3]:
# Load environment variables
_ = load_dotenv()
working_dir = os.getenv("WORKING_DIR")

# Change to working dir
os.chdir(working_dir)

In [4]:
def extract_symbols():
    exchange = ccxt.binance(
        {
            "rateLimit": 2000,
            "enableRateLimit": True,
            "verbose": False,
        }
    )
    data = exchange.load_markets()
    df = pd.DataFrame(data).T.reset_index()
    df = df.drop(columns=['precision','limits','info'])
    return df

In [38]:
df_exchange = extract_symbols()

In [48]:
df_exchange.active = df_exchange.active.astype(int)

In [46]:
# Write data to database
with sqlite3.connect(database="./data/database.db") as conn:
    df_database = pd.read_sql("SELECT pk_symbols, id, baseId, quoteId, type, active as last_status FROM DimSymbols", conn)
    
df_database.last_status = df_database.last_status.astype(int)
    

In [49]:
df_new = df_exchange.merge(df_database, on=['id','baseId','quoteId', "type"], how='outer')


In [56]:
df_insert = df_new[df_new.pk_symbols.isna()]
if not df_insert.empty:
    with sqlite3.connect(database="./data/database.db") as conn:
        df_insert.to_sql("DimSymbols", conn, if_exists='append', index=False)
    

In [63]:
df_update = df_new[(df_new.active != df_new.last_status)]

if not df_update.empty:
    symbols2update = df_update.pk_symbols.to_list()
    with sqlite3.connect(database="./data/database.db") as conn:
        query = f"""
        update DimSymbols
        set active = 0
        where pk_symbols in ({symbols2update})
        """
        df_update.pk_symbols.to_list()